In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import re
import numpy as np

In [33]:
ifn_biomass_fnames = Path('ifn4_biomass/').glob('*.shp')

savedir = 'ifn4_biomass_flights/'
if not Path(savedir).exists():
        Path(savedir).mkdir(parents=True)


pnoa_flights = ['1','2']
for fname in ifn_biomass_fnames:
    
    ifn4 = gpd.read_file(fname)
    utm = re.findall('ifn4_(.*)_biomass',fname.stem)[0]

    ifn4['pnoa_file'] = None
    for nflight in pnoa_flights:
        flight_datadir = 'Entornos_Vuelo_LiDAR/{}_Cobertura/Huso_{}/'.format(nflight, utm)
        flight_fname = next(Path(flight_datadir).glob('*.shp'))
        flight = gpd.read_file(flight_fname)
        flight = flight.to_crs('EPSG:258{}'.format(utm))

        for _, row in flight.iterrows():
            # print(row)
            
            try:
                year_str = row.FECHA
            except:
                year_str = row.A_OVUELO
            try:
                year_list = year_str.split(', ')
                if len(year_list[0]) > 4:
                    year_list = year_str.split(',')
            except:
                continue
            
            year_gap = np.max([int(y) for y in year_list]) - np.min([int(y) for y in year_list])

            if year_gap>1:
                continue
            
            if '.asc' in row.FICHERO:
                rasterfile = '{}{}'.format(re.findall("(NDSM.*).asc",row.FICHERO)[0],'.tif') 
            elif '.tif' in row.FICHERO:  
                rasterfile = row.FICHERO
            else: 
                continue     
            ifn4['pnoa_file'] = np.where( ifn4.geometry.within(row.geometry) & (ifn4.Year.isin(year_list)), rasterfile, ifn4['pnoa_file'])

            
    ifn4 = ifn4.dropna()
    ifn4['mission'] = ifn4.apply(lambda row: re.findall("-COB(.)",row.pnoa_file)[0] , axis=1) 
    ifn4.to_file('{}ifn4_{}_biomass_flight.shp'.format(savedir,utm), driver='ESRI Shapefile')